# 船の座標と目標点の

In [2]:
import pandas as pd

import sys
sys.path.append("/Users/hashiguchiyutaka/Desktop/3S/prog/submarine-py")

from lib.make_coordinates import make_all_coordinates, valid_coordinates, make_not_near_coordinates
from lib.player_base import Player, PlayerShip


In [3]:
class Enemy:
    def __init__(self):
        self.df = pd.DataFrame(make_all_coordinates()) # 盤面の全パターンを初期値に
        # self.hp = {"w": 3, "c": 2, "s": 1} 相手のhpは今のところ不要では？
        self.ships = ["w", "c", "s"]

    def remove_same_position(self):
        """
        self.ships内の船の位置が重複している行を削除する
        """
        # 船が1つしかなければ重複はない
        if len(self.ships) == 1:
            return None
        elif len(self.ships) == 2:
            self.df = self.df[self.df.apply(lambda x: x[self.ships[0]] != x[self.ships[1]], axis=1)]
        else:
            self.df = self.df[self.df.apply(lambda x: x["w"] != x["c"] and x["w"] != x["s"] and x["c"] != x["s"], axis=1)]

        # 0になったらself.shipsからその船を消す
        # 直接消してしまうと、remove_same_position(self.df)などの実装で不具合があるので、probを求める際に消えている船の座標はカウントしないことにする
        # と思ったけど、こうすると、remove_same_positionで、死んだ船と被っている行動も不可能と見做されてしまう
        # 他の船の位置の組に重複があるが、確率を考える上では問題ない

    def move(self, ship_type, direction) -> None:
        """
        敵のmoveを処理し、self.dfを更新
        direction = [1,0]などの移動分を表すベクトル
        self.df.loc[:, ship_type]を使うことでSettingWithCopyWarningをさけ、確実に元のdfを変更できるようにする
        """
        delta_x = direction[0]
        delta_y = direction[1]
        self.df.loc[:, ship_type] = self.df[ship_type].map(lambda x: (x[0] + delta_x , x[1] + delta_y))
        
        
        # 有効な座標以外の盤面を削除
        self.df = self.df[self.df[ship_type].map(lambda x: x in valid_coordinates())]

        # 座標がダブっている船を含む盤面を削除(移動によりダブってしまうことがある)
        self.remove_same_position()
    
    def attack(self, position) -> None:
        """
        敵のattackを処理し、self.dfを更新
        position = [1,0]などの相手の攻撃した座標
        """
        # positionの周囲1マスを取得(ここに相手がいることになる)
        # ここでは盤面の外の座標が入ってしまっても問題ない
        near_positions = [(position[0] + i, position[1] + j) for i in range(-1, 2) for j in range(-1, 2)]

        # self.dfの中から、near_positionsに含まれる座標を持つ行のみを残す
        # ただし、w,c,sが全て存在するとは限らないことに注意

        # self.dfの行が、near_positionsに含まれる座標を持つかどうかを判定する関数
        def is_valid(x): # xはself.dfの行
            for ship in self.ships:
                if x[ship] in near_positions:
                    return True
            # どの船もnear_positionsに含まれる座標を持っていない場合
            return False
            
        self.df = self.df[self.df.apply(is_valid, axis=1)]

    def hit(self, ship_type, position) -> None:
        """
        自分の攻撃がhitして、かつその船がまだ生きている時の処理
        """
        position = tuple(position) # 忘れずに
        self.df = self.df[self.df[ship_type] == position]
        # print(self.prob()) hit関数が動いているなら、ここを表示すると常に確率1の座標が存在するはず

    def miss(self, position) -> None:
        """
        自分の攻撃がmissした時の処理
        """
        # 結局、near/not_nearで処理するので何もしない
        pass

    def near(self, ship_type, position) -> None:
        """
        自分の攻撃の結果、ship_typeがnearにいた時の処理
        positionの周囲1マス(自身を含まない)にその船がいることが確定する
        """
        position = tuple(position)
        # near_positionsはpositionの周囲1マス(自身を含まない)の座標
        near_positions = [(position[0] + i, position[1] + j) for i in range(-1, 2) for j in range(-1, 2)]
        near_positions.remove(position) 
        self.df = self.df[self.df[ship_type].map(lambda x: x in near_positions)]

    def not_near(self, ship_type, position) -> None:
        """ 
        自分の攻撃の結果、ship_typeがnot_nearだった時の処理
        positionの周囲1マス(自身を含む)以外のマスにその船がいることが確定する
        """
        position = tuple(position)
        # near_positionsはpositionの周囲1マス(自身を含む)の座標
        near_positions = [(position[0] + i, position[1] + j) for i in range(-1, 2) for j in range(-1, 2)]
        self.df = self.df[self.df[ship_type].map(lambda x: x not in near_positions)]

    def prob(self):
        return self.df.stack().value_counts()/len(self.df)
    
    def where_to_attack(self):
        """
        self.dfの中から、確率高い座標を順に返す
        なお、enemy クラス内部の処理ではdictのkeyにできるという理由でtupleを使っていたが、ここでlistに戻す
        [[4, 2],[4, 0],...]
        """
        return [list(coordinate) for coordinate in self.prob().keys().tolist()]

In [5]:
enemy = Enemy()
dict(enemy.prob())

{(4, 0): 0.12,
 (1, 2): 0.12,
 (1, 4): 0.12,
 (2, 0): 0.12,
 (0, 3): 0.12,
 (1, 1): 0.12,
 (0, 0): 0.12,
 (4, 4): 0.12,
 (4, 1): 0.12,
 (3, 2): 0.12,
 (2, 1): 0.12,
 (0, 4): 0.12,
 (2, 4): 0.12,
 (3, 4): 0.12,
 (0, 1): 0.12,
 (3, 3): 0.12,
 (3, 0): 0.12,
 (4, 2): 0.12,
 (1, 3): 0.12,
 (1, 0): 0.12,
 (2, 2): 0.12,
 (0, 2): 0.12,
 (3, 1): 0.12,
 (4, 3): 0.12,
 (2, 3): 0.12}